In [2]:
from rlcms import composites
import ee
import geemap

ee.Initialize()

ModuleNotFoundError: No module named 'rlcms'

In [2]:
# some args we'll use in both composites
start_date = '2018-01-01'
end_date = '2018-12-31'
reducer = 'mean' # min, max, ee.Reducer objects
composite_mode = 'annual' # 'annual'

sesheke = ee.FeatureCollection("projects/wwf-sig/assets/kaza-lc/aoi/testSeshekeWater")
# binga_polys = ee.FeatureCollection("projects/wwf-sig/assets/kaza-lc/reference_data/BingaDummyReferencePolys")
ca_poly = ee.Geometry.Polygon([[-120.70162384826843,36.658181801020476],
                               [-120.65218537170593,36.658181801020476],
                               [-120.65218537170593,36.68461734040718],
                               [-120.70162384826843,36.68461734040718],
                               [-120.70162384826843,36.658181801020476]])

s1composite = composites.composite(dataset='Sentinel1',
                                  region=sesheke,
                                  start_date=start_date,
                                  end_date=end_date,
                                 composite_mode=composite_mode,
                                 reducer=reducer)
s2composite = composites.composite(dataset='Sentinel2',
                                 region=sesheke,
                                 start_date=start_date,
                                 end_date=end_date,
                                 indices=['IBI','SAVI'],
                                 composite_mode=composite_mode,
                                 reducer=reducer)

print(s1composite.__dict__)
print(s1composite.bands)
print(s2composite.bands)

all_composite = s1composite.stack(s2composite)

print(all_composite.bandNames().getInfo())

{'dataset': 'Sentinel1', 'region': [[[24.280821946182392, -17.49195652431585], [24.31515256390456, -17.49195647837019], [24.31515256390456, -17.467721268955287], [24.280821946182392, -17.467721310351774], [24.280821946182392, -17.49195652431585]]], 'start_date': '2018-01-01', 'end_date': '2018-12-31', 'composite_mode': 'annual', 'reducer': 'mean', 'bands': ['VV_mean', 'VH_mean', 'angle_mean'], 'composite': <ee.image.Image object at 0x000002363CBE0AC0>}
['VV_mean', 'VH_mean', 'angle_mean']
['blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'swir1_mean', 'swir2_mean', 'IBI_mean', 'SAVI_mean']
['Sentinel1_VV_mean', 'Sentinel1_VH_mean', 'Sentinel1_angle_mean', 'Sentinel2_blue_mean', 'Sentinel2_green_mean', 'Sentinel2_red_mean', 'Sentinel2_nir_mean', 'Sentinel2_swir1_mean', 'Sentinel2_swir2_mean', 'Sentinel2_IBI_mean', 'Sentinel2_SAVI_mean']


In [4]:
Map = geemap.Map()

s2_params = {'bands':['red_mean','blue_mean','green_mean'],
              'min':0,'max':1500,
              }
s2_params_multi_year = {'bands':['t0_red_mean','t0_blue_mean','t0_green_mean'],
              'min':0,'max':1500,
              }
s1_params = {'bands':'VV_mean','min':-22,'max':2}
s1_params_multi_year = {'bands':'t0_VV_mean','min':-22,'max':2}
# Map.addLayer(binga_polys,{},'binga pols FC')
Map.addLayer(ca_poly,{},'single poly geo')
# Map.addLayer(colombia_polys,{},'multi poly geometry')
Map.addLayer(all_composite,s2_params_multi_year,'all_composite rgb')
Map.addLayer(all_composite,s1_params_multi_year,'all_composite VV')
Map.addLayer(s1composite,s1_params_multi_year,'s1composite VV')

Map.centerObject(ca_poly,12)
# Map.centerObject(colombia_polys,12)
# Map.centerObject(binga_polys,12)


Map

Map(center=[36.671401368184334, -120.67690460998617], controls=(WidgetControl(options=['position', 'transparen…